In [39]:

import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import gspread

In [78]:
from utils import FTP_Connection
import yaml
CONFIG_PATH = '../config.yaml'

with open(CONFIG_PATH, 'r', encoding='utf-8') as file:
    scraper_config = yaml.safe_load(file)

conn = FTP_Connection(scraper_config['sources']['ftp']['host'])
print('\n'.join(conn.ls('ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099')))


ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 1.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 10.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 11.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 12.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 13.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 14.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 15.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 16.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 18.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 2.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 20.xml
ftp://ftp.legis.state.tx.us/bills/

In [41]:
def get_bill_history_df(conn, scraper_config):
    """
    Get bill history data from FTP server and return as DataFrame
    
    Args:
        conn: FTP_Connection object
        scraper_config: Dictionary containing scraper configuration
        
    Returns:
        DataFrame containing bill history data
    """
    # Build URL from config
    base_url = scraper_config['sources']['ftp']['base_path'].format(
        LegSess=scraper_config['info']['LegSess']
    )
    history_url = f"{base_url}/billhistory/history.xml"
    
    # Get and parse XML
    bill_history_xml = conn.get_data(history_url)
    bill_history_soup = BeautifulSoup(bill_history_xml, 'xml')
        
    # Get bills element
    bills = bill_history_soup.find('bills')

    # Convert bills to list of dicts
    bill_list = []
    for bill in bills.find_all('bill'):
        bill_dict = {
            'id': bill.get('id'),
            'path': bill.get('path'), 
            'timestamp': bill.get('timestamp')
        }
        bill_list.append(bill_dict)
    return pd.DataFrame(bill_list)

bill_df = get_bill_history_df(conn, scraper_config)
bill_df.to_csv('raw_data/bill_history.csv', index=False)


In [42]:
def get_bill_urls(conn, scraper_config):
    """
    Get list of URLs for all bill XML files in house_bills and senate_bills directories.
    
    Args:
        conn: FTP_Connection object
        scraper_config: Dictionary containing scraper configuration
        
    Returns:
        List of URLs for all bill XML files
    """
    # Build base URL from config
    base_url = scraper_config['sources']['ftp']['base_path'].format(
        LegSess=scraper_config['info']['LegSess']
    )
    
    bill_urls = []
    
    # Process both house and senate bills
    for chamber in ['house_bills', 'senate_bills']:
        # Build URL for this chamber
        chamber_url = f"{base_url}/billhistory/{chamber}"
        
        # Get list of bill range folders (HB00001_HB00099 etc)
        range_folders = conn.ls(chamber_url)
        
        # Get bill XML files from each range folder
        for folder_url in range_folders:
            bill_xmls = conn.ls(folder_url)
            bill_urls.extend(bill_xmls)
            
    return bill_urls

# Get all bill URLs
# bill_urls = get_bill_urls(conn, scraper_config)
# print(f"Found {len(bill_urls)} total bill XML files")
# print("\nFirst 5 bill URLs:")
# print('\n'.join(bill_urls[:5]))


In [68]:

def investigate_xml(url, conn):
    """
    Get XML data from a URL, prettify it, and save it to raw_data directory with _raw suffix
    
    Args:
        url: URL to retrieve XML from
        conn: FTP_Connection object
    """
    # Get XML data
    xml_data = conn.get_data(url)
    
    if xml_data:
        # Parse and prettify XML
        soup = BeautifulSoup(xml_data, 'xml')
        pretty_xml = soup.prettify()
        
        # Generate filename from URL
        filename = url.split('/')[-1].replace('.xml', '_raw.xml')
        
        # Save to raw_data directory
        with open(f'raw_data/{filename}', 'w', encoding='utf-8') as f:
            f.write(pretty_xml)
        print(f"Saved {url} to raw_data/{filename}")
    else:
        print(f"Failed to retrieve data from {url}")

# Test the function with a bill history URL
test_url = "ftp://ftp.legis.state.tx.us/bills/89R/billhistory/senate_bills/SB00001_SB00099/SB 2.xml"
investigate_xml(test_url, conn)



Saved ftp://ftp.legis.state.tx.us/bills/89R/billhistory/senate_bills/SB00001_SB00099/SB 2.xml to raw_data/SB 2_raw.xml


In [65]:
import json


ftp://ftp.legis.state.tx.us/bills/88R/billhistory/house_bills/HB00001_HB00099/HB 2.xml
Parsed bill data:
[
  {
    "type": "house",
    "name": "Ways & Means",
    "status": "Out of committee",
    "votes": {
      "aye": 10,
      "nay": 1,
      "absent": 0,
      "present_not_voting": 0
    }
  },
  {
    "type": "senate",
    "name": "Local Government",
    "status": "In committee",
    "votes": {
      "aye": 0,
      "nay": 0,
      "absent": 0,
      "present_not_voting": 0
    }
  }
]


In [45]:
def get_raw_bills_data(scraper_config, ftp_connection):
    bill_urls = get_bill_urls(ftp_connection, scraper_config)
    raw_bills = []
    for url in bill_urls:
        bill_data = parse_bill_xml(ftp_connection, url)
        if bill_data:
            raw_bills.append(bill_data)
    return pd.DataFrame(raw_bills)

raw_bills_df = get_raw_bills_data(scraper_config, conn)
raw_bills_df

ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 1.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 10.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 11.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 12.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 13.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 14.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 15.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 2.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 20.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 21.xml
ftp://ftp.legis.state.tx.us/bills/89R/billhistory/house_bills/HB00001_HB00099/HB 22.xml
ftp://ftp.legis.state.tx.us/bills/

,bill_id,last_update,last_action,caption,caption_version,authors,coauthors,sponsors,cosponsors,subjects,companions,committees,actions,versions
0,89(R) HB 1,3/4/2025 2:00:03 AM,02/25/2025 H Referred to Appropriations: Feb 2...,General Appropriations Bill.,Introduced,[Bonnen],[],[],[],[State Finances--Appropriations (I0746)],[],"[{'type': 'house', 'name': 'Appropriations', '...","[{'number': 'H001', 'date': '1/22/2025', 'desc...","[{'type': 'Bill', 'description': 'Introduced',..."
1,89(R) HB 10,3/4/2025 2:00:03 AM,03/03/2025 H Referred to Delivery of Governmen...,Relating to reforming the procedure by which s...,Introduced,[Capriglione],[],[],[],"[Business & Commerce--General (I0050), Courts-...",[],"[{'type': 'house', 'name': 'Delivery of Govern...","[{'number': 'H001', 'date': '2/27/2025', 'desc...","[{'type': 'Bill', 'description': 'Introduced',..."
2,89(R) HB 11,3/4/2025 2:00:03 AM,03/03/2025 H Referred to Licensing & Administr...,Relating to occupational licensing reciprocity...,Introduced,[Phelan],[],[],[],"[Intergovernmental Relations (I0447), Occupati...",[],"[{'type': 'house', 'name': 'Licensing & Admini...","[{'number': 'H001', 'date': '2/27/2025', 'desc...","[{'type': 'Bill', 'description': 'Introduced',..."
3,89(R) HB 12,3/6/2025 12:00:02 PM,03/06/2025 H Referred to Delivery of Governmen...,Relating to the review and audit of certain st...,Introduced,"[Bell, Keith]",[],[],[],"[State Agencies, Boards & Commissions (I0749),...","[{'bill_id': 'SB 1943', 'author': 'Parker', 'r...","[{'type': 'house', 'name': 'Delivery of Govern...","[{'number': 'H001', 'date': '3/6/2025', 'descr...","[{'type': 'Bill', 'description': 'Introduced',..."
4,89(R) HB 13,3/6/2025 1:00:05 PM,03/06/2025 H Filed,Relating to creating the Texas Interoperabilit...,Introduced,[King],[],[],[],"[Disaster Preparedness & Relief (I0211), Gover...",[],"[{'type': 'house', 'name': '', 'status': '', '...","[{'number': 'H001', 'date': '3/6/2025', 'descr...","[{'type': 'Bill', 'description': 'Introduced',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5877,89(R) SB 2021,3/6/2025 6:00:11 PM,03/06/2025 S Filed,Relating to the interconnection and integratio...,Introduced,[Johnson],[],[],[],"[Energy--General (I0315), Utilities--Electric ...",[],"[{'type': 'house', 'name': '', 'status': '', '...","[{'number': 'S000', 'date': '3/6/2025', 'descr...","[{'type': 'Bill', 'description': 'Introduced',..."
5878,89(R) SB 2022,3/6/2025 6:00:11 PM,03/06/2025 S Filed,Relating to the denial of the registration of ...,Introduced,[Blanco],[],[],[],"[County Government--Finance (I0097), County Go...",[],"[{'type': 'house', 'name': '', 'status': '', '...","[{'number': 'S000', 'date': '3/6/2025', 'descr...","[{'type': 'Bill', 'description': 'Introduced',..."
5879,89(R) SB 2023,3/6/2025 6:00:11 PM,03/06/2025 S Filed,Relating to the establishment of a grant progr...,Introduced,[Blanco],[],[],[],"[Cemeteries (I0054), County Government--Genera...",[],"[{'type': 'house', 'name': '', 'status': '', '...","[{'number': 'S000', 'date': '3/6/2025', 'descr...","[{'type': 'Bill', 'description': 'Introduced',..."
5880,89(R) SB 2024,3/6/2025 6:00:11 PM,03/06/2025 S Filed,"Relating to a prohibition on marketing, advert...",Introduced,[Perry],[],[],[],"[Health--General (I0385), Safety (I0740), Toba...",[],"[{'type': 'house', 'name': '', 'status': '', '...","[{'number': 'S000', 'date': '3/6/2025', 'descr...","[{'type': 'Bill', 'description': 'Introduced',..."


In [52]:
def clean_bill_id(bill_id):
    """
    Transform bill ID from format like '89(R) HB 1' into standardized format.
    
    Args:
        bill_id (str): Bill ID in format like '89(R) HB 1'
        
    Returns:
        tuple: (bill_number, session)
            bill_number (str): Bill number in format like 'HB1'
            session (str): Session in format like '89R'
    """
    # Split into session and bill parts
    session_part, bill_part = bill_id.split(') ')
    
    # Clean session (e.g. '89(R' -> '89R')
    session = session_part.replace('(', '')
    
    # Clean bill number (e.g. 'HB 1' -> 'HB1')
    bill_number = bill_part.replace(' ', '')
    
    return bill_number, session

def get_authors_data(raw_bills_df):
    authors_data = []
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        for author in row['authors']:
            authors_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'author': author,
                'author_type': 'Author'
            })
        for coauthor in row['coauthors']:
            authors_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'author': coauthor,
                'author_type': 'coauthor'
            })
    return pd.DataFrame(authors_data, columns=['bill_id', 'leg_id', 'author', 'author_type'])
authors_df = get_authors_data(raw_bills_df)
authors_df.to_csv('raw_data/authors.csv', index=False)
authors_df


,bill_id,leg_id,author,author_type
0,HB1,89R,Bonnen,Author
1,HB10,89R,Capriglione,Author
2,HB11,89R,Phelan,Author
3,HB12,89R,"Bell, Keith",Author
4,HB13,89R,King,Author
...,...,...,...,...
7489,SB2021,89R,Johnson,Author
7490,SB2022,89R,Blanco,Author
7491,SB2023,89R,Blanco,Author
7492,SB2024,89R,Perry,Author


In [53]:
def get_sponsors_data(raw_bills_df):
    """
    Extract sponsors data from raw bills dataframe into standardized format.
    
    Args:
        raw_bills_df (pd.DataFrame): DataFrame containing raw bill data
        
    Returns:
        pd.DataFrame: DataFrame with columns bill_id, leg_id, sponsor, sponsor_type
    """
    sponsors_data = []
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        for sponsor in row['sponsors']:
            sponsors_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id, 
                'sponsor': sponsor,
                'sponsor_type': 'Sponsor'
            })
        for cosponsor in row['cosponsors']:
            sponsors_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'sponsor': cosponsor,
                'sponsor_type': 'cosponsor'
            })
    return pd.DataFrame(sponsors_data, columns=['bill_id', 'leg_id', 'sponsor', 'sponsor_type'])

sponsors_df = get_sponsors_data(raw_bills_df)
sponsors_df.to_csv('raw_data/sponsors.csv', index=False)
sponsors_df


,bill_id,leg_id,sponsor,sponsor_type


In [54]:
def get_subjects_data(raw_bills_df):
    """
    Extract subjects data from raw bills dataframe into standardized format.
    Splits subject strings into title and ID components.
    
    Args:
        raw_bills_df (pd.DataFrame): DataFrame containing raw bill data
        
    Returns:
        pd.DataFrame: DataFrame with columns bill_id, leg_id, subject_title, subject_id
    """
    subjects_data = []
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        for subject in row['subjects']:
            # Split subject into title and ID
            # Example: "City Government--Employees/Officers (I0061)"
            title = subject.split(' (')[0]  # Everything before the ID
            subject_id = subject.split('(')[1].rstrip(')')  # Extract ID without parentheses
            
            subjects_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'subject_title': title,
                'subject_id': subject_id
            })
    return pd.DataFrame(subjects_data, columns=['bill_id', 'leg_id', 'subject_title', 'subject_id'])

subjects_df = get_subjects_data(raw_bills_df)
subjects_df.to_csv('raw_data/subjects.csv', index=False)
subjects_df

,bill_id,leg_id,subject_title,subject_id
0,HB1,89R,State Finances--Appropriations,I0746
1,HB10,89R,Business & Commerce--General,I0050
2,HB10,89R,Courts--General,I0160
3,HB10,89R,Governor,I0375
4,HB10,89R,"State Agencies, Boards & Commissions",I0749
...,...,...,...,...
26016,SB2023,89R,State Finances--Management & Control,I0748
26017,SB2023,89R,HEALTH & HUMAN SERVICES COMMISSION,V0177
26018,SB2024,89R,Health--General,I0385
26019,SB2024,89R,Safety,I0740


In [56]:
def get_companions_data(raw_bills_df):
    """
    Extract companion bill relationships from raw bills dataframe into standardized format.
    
    Args:
        raw_bills_df (pd.DataFrame): DataFrame containing raw bill data
        
    Returns:
        pd.DataFrame: DataFrame with columns bill_id, leg_id, companion_bill_id
    """
    companions_data = []
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        
        # Skip if no companions
        if not row['companions']:
            continue
            
        for companion in row['companions']:
            companions_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'companion_bill_id': companion['bill_id'],
                'relationship': companion['relationship']
            })
            
    return pd.DataFrame(companions_data, columns=['bill_id', 'leg_id', 'companion_bill_id', 'relationship'])

companions_df = get_companions_data(raw_bills_df)
companions_df.to_csv('raw_data/companions.csv', index=False)
companions_df


,bill_id,leg_id,companion_bill_id,relationship
0,HB12,89R,SB 1943,Identical
1,HB15,89R,SB 29,Identical
2,HB21,89R,SB 867,Identical
3,HB23,89R,HB 2977,Duplicate
4,HB24,89R,SB 844,Identical
...,...,...,...,...
1287,SB2004,89R,HB 3962,Identical
1288,SB2007,89R,HB 3531,Identical
1289,SB2009,89R,HB 1445,Identical
1290,SB2018,89R,HB 3232,Identical


In [58]:
def get_actions_data(raw_bills_df):
    """
    Extract action data from raw bills dataframe into standardized format.
    
    Args:
        raw_bills_df (pd.DataFrame): DataFrame containing raw bill data
        
    Returns:
        pd.DataFrame: DataFrame with action information
    """
    actions_data = []
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        
        # Skip if no actions
        if not row['actions']:
            continue
            
        for action in row['actions']:
            actions_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'actionNumber': action['number'],
                'action_date': action['date'],
                'description': action['description'],
                'comment': action['comment'],
                'action_timestamp': action.get('timestamp', None)
            })
            
    return pd.DataFrame(actions_data, columns=['bill_id', 'leg_id', 'actionNumber', 'action_date', 
                                             'description', 'comment', 'action_timestamp'])

actions_df = get_actions_data(raw_bills_df)
actions_df.to_csv('raw_data/actions.csv', index=False)
actions_df


,bill_id,leg_id,actionNumber,action_date,description,comment,action_timestamp
0,HB1,89R,H001,1/22/2025,Filed,,
1,HB1,89R,H005,2/25/2025,Read first time,,
2,HB1,89R,H015,2/25/2025,Referred to Appropriations,,2/25/2025 3:35:00 PM
3,HB10,89R,H001,2/27/2025,Filed,,
4,HB10,89R,H005,3/3/2025,Read first time,,
...,...,...,...,...,...,...,...
14003,SB2023,89R,S001,3/6/2025,Filed,,
14004,SB2024,89R,S000,3/6/2025,Received by the Secretary of the Senate,,
14005,SB2024,89R,S001,3/6/2025,Filed,,
14006,SB2025,89R,S000,3/6/2025,Received by the Secretary of the Senate,,


In [66]:
def get_committees_data(raw_bills_df):
    """
    Extract committee data from raw bills dataframe into standardized format.
    
    Args:
        raw_bills_df (pd.DataFrame): DataFrame containing raw bill data
        
    Returns:
        pd.DataFrame: DataFrame with committee information
    """
    committees_data = []
    
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        
        # Skip if no committees
        if not row['committees']:
            continue
            
        for committee in row['committees']:
            votes = committee.get('votes', {})
            committees_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'chamber': committee.get('type'),
                'name': committee.get('name'),
                'subcommittee_name': None,  # Not in sample data but included for schema
                'status': committee.get('status'),
                'subcommittee_status': None,  # Not in sample data but included for schema
                'aye_votes': votes.get('aye', 0),
                'nay_votes': votes.get('nay', 0), 
                'present_votes': votes.get('present_not_voting', 0),
                'absent_votes': votes.get('absent', 0)
            })
            
    return pd.DataFrame(committees_data, columns=['bill_id', 'leg_id', 'chamber', 'name',
                                                'subcommittee_name', 'status', 'subcommittee_status',
                                                'aye_votes', 'nay_votes', 'present_votes', 'absent_votes'])

committees_df = get_committees_data(raw_bills_df)
committees_df.to_csv('raw_data/committees.csv', index=False)
committees_df


,bill_id,leg_id,chamber,name,subcommittee_name,status,subcommittee_status,aye_votes,nay_votes,present_votes,absent_votes
0,HB1,89R,house,Appropriations,None,In committee,None,0,0,0,0
1,HB1,89R,senate,,None,,None,0,0,0,0
2,HB10,89R,house,Delivery of Government Efficiency,None,In committee,None,0,0,0,0
3,HB10,89R,senate,,None,,None,0,0,0,0
4,HB11,89R,house,Licensing & Administrative Procedures,None,In committee,None,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
11759,SB2023,89R,senate,,None,,None,0,0,0,0
11760,SB2024,89R,house,,None,,None,0,0,0,0
11761,SB2024,89R,senate,,None,,None,0,0,0,0
11762,SB2025,89R,house,,None,,None,0,0,0,0


In [64]:
def get_versions_data(raw_bills_df):
    versions_data = []
    
    for _, row in raw_bills_df.iterrows():
        bill_id, leg_id = clean_bill_id(row['bill_id'])
        
        # Skip if no versions
        if not row['versions']:
            continue
            
        for version in row['versions']:
            # Extract URLs, defaulting to None if not present
            urls = version.get('urls', {})
            versions_data.append({
                'bill_id': bill_id,
                'leg_id': leg_id,
                'type': version.get('type'),
                'description': version.get('description'),
                'html_url': urls.get('web_html'),
                'pdf_url': urls.get('web_pdf'),
                'ftp_html_url': urls.get('ftp_html'),
                'ftp_pdf_url': urls.get('ftp_pdf')
            })
            
    return pd.DataFrame(versions_data, columns=['bill_id', 'leg_id', 'type', 'description',
                                              'html_url', 'pdf_url', 'ftp_html_url', 'ftp_pdf_url'])

versions_df = get_versions_data(raw_bills_df)
versions_df.to_csv('raw_data/versions.csv', index=False)
versions_df


,bill_id,leg_id,type,description,html_url,pdf_url,ftp_html_url,ftp_pdf_url
0,HB1,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
1,HB10,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
2,HB11,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
3,HB12,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
4,HB13,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
...,...,...,...,...,...,...,...,...
6171,SB2020,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
6172,SB2021,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
6173,SB2022,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...
6174,SB2023,89R,Bill,Introduced,http://capitol.texas.gov/tlodocs/89R/billtext/...,http://capitol.texas.gov/tlodocs/89R/billtext/...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...,ftp://ftp.legis.state.tx.us/bills/89R/billtext...


In [8]:
# Compare history.xml and history_periodic.xml
history_url = "ftp://ftp.legis.state.tx.us/bills/89R/billhistory/history.xml"
periodic_url = "ftp://ftp.legis.state.tx.us/bills/89R/billhistory/history_periodic.xml"

# Get both files
investigate_xml(history_url, conn)
investigate_xml(periodic_url, conn)

# Load and parse both files
with open('raw_data/history_raw.xml', 'r', encoding='utf-8') as f:
    history_soup = BeautifulSoup(f.read(), 'xml')
    
with open('raw_data/history_periodic_raw.xml', 'r', encoding='utf-8') as f:
    periodic_soup = BeautifulSoup(f.read(), 'xml')

# Get all bill IDs and timestamps from each file
history_bills = {b['id']: b['timestamp'] for b in history_soup.find_all('bill')}
periodic_bills = {b['id']: b['timestamp'] for b in periodic_soup.find_all('bill')}

# Find bills unique to each file
only_in_history = set(history_bills.keys()) - set(periodic_bills.keys())
only_in_periodic = set(periodic_bills.keys()) - set(history_bills.keys())

print("Differences between history.xml and history_periodic.xml:")
print(f"\nNumber of bills in history.xml: {len(history_bills)}")
print(f"Number of bills in history_periodic.xml: {len(periodic_bills)}")

print(f"\nBills only in history.xml: {len(only_in_history)}")
if only_in_history:
    print("Examples:", list(only_in_history)[:5])
    
print(f"\nBills only in history_periodic.xml: {len(only_in_periodic)}")
if only_in_periodic:
    print("Examples:", list(only_in_periodic)[:5])

# Compare timestamps for bills in both files
common_bills = set(history_bills.keys()) & set(periodic_bills.keys())
different_timestamps = {
    bill: (history_bills[bill], periodic_bills[bill])
    for bill in common_bills
    if history_bills[bill] != periodic_bills[bill]
}

print(f"\nBills with different timestamps: {len(different_timestamps)}")
if different_timestamps:
    print("\nExample timestamp differences:")
    for bill, (hist_time, per_time) in list(different_timestamps.items())[:5]:
        print(f"{bill}:")
        print(f"  history.xml: {hist_time}")
        print(f"  periodic.xml: {per_time}")




Saved ftp://ftp.legis.state.tx.us/bills/89R/billhistory/history.xml to raw_data/history_raw.xml
Saved ftp://ftp.legis.state.tx.us/bills/89R/billhistory/history_periodic.xml to raw_data/history_periodic_raw.xml
Differences between history.xml and history_periodic.xml:

Number of bills in history.xml: 724
Number of bills in history_periodic.xml: 787

Bills only in history.xml: 544
Examples: ['HB 3855', 'HB 24', 'HB 806', 'HB 3861', 'SB 1920']

Bills only in history_periodic.xml: 607
Examples: ['HB 957', 'SB 1366', 'HR 428', 'SB 1514', 'SJR 66']

Bills with different timestamps: 180

Example timestamp differences:
HR 50:
  history.xml: 3/6/2025 2:00 AM
  periodic.xml: 3/6/2025 2:00 PM
HCR 26:
  history.xml: 3/6/2025 2:00 AM
  periodic.xml: 3/6/2025 12:00 PM
HR 142:
  history.xml: 3/6/2025 2:00 AM
  periodic.xml: 3/6/2025 12:00 PM
HR 149:
  history.xml: 3/6/2025 2:00 AM
  periodic.xml: 3/6/2025 12:00 PM
HR 212:
  history.xml: 3/6/2025 2:00 AM
  periodic.xml: 3/6/2025 2:00 PM


In [ ]:

def extract_committee_meetings_links(committees_page_url, leg_id):
    session = requests.Session()

    # Step 1: GET request to retrieve hidden form fields
    response = session.get(committees_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract necessary hidden fields
    viewstate = soup.find("input", {"name": "__VIEWSTATE"})["value"]
    eventvalidation = soup.find("input", {"name": "__EVENTVALIDATION"})["value"]
    viewstategenerator = soup.find("input", {"name": "__VIEWSTATEGENERATOR"})["value"]

    # Step 2: POST request with hidden fields and selected legislature
    data = {
        "__VIEWSTATE": viewstate,
        "__EVENTVALIDATION": eventvalidation,
        "__VIEWSTATEGENERATOR": viewstategenerator,
        "__EVENTTARGET": "ddlLegislature",  # Mimic dropdown change
        "__EVENTARGUMENT": "",
        "ddlLegislature": leg_id
    }

    response = session.post(committees_page_url, data=data)

    soup = BeautifulSoup(response.text, 'html.parser')

    committees_list = soup.find_all('a', id='CmteList')

    committees = []

    for committee in committees_list:
        committees.append({
            'name': committee.text.strip(),
            'href': committee['href']
        })

    return committees

def get_committee_meetings_links(committees_page_url, meetings_by_committee_url, leg_id):

    for chamber in ['H', 'J', 'S']:
        committees_page_url = f"{committees_page_url}?Chamber={chamber}"
        committees = extract_committee_meetings_links(committees_page_url, leg_id)

        committee_meetings = []
        for committee in committees:
            committee_meetings.append({
                'name': committee['name'],
                'href': meetings_by_committee_url + committee['href'],
                'chamber': chamber,
                'leg_id': leg_id
            })
        return pd.DataFrame(committee_meetings)

committees_url = "https://capitol.texas.gov/Committees/Committees.aspx"
meetings_by_committee_url = 'https://capitol.texas.gov/Committees/'
committee_meetings_df = get_committee_meetings_links(committees_url,meetings_by_committee_url, "89")  # Get 88th Legislature data
committee_meetings_df


https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C020
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C030
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C010
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C012
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C013
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C014
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C050
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C200
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C220
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C221
https://capitol.texas.gov/Committees/MeetingsByCmte.aspx?Leg=89&Chamber=H&CmteCode=C430
https://capitol.texas.gov/Commit

,name,href,chamber,leg_id
0,Agriculture & Livestock,https://capitol.texas.gov/Committees/MeetingsB...,H,89
1,Appropriations,https://capitol.texas.gov/Committees/MeetingsB...,H,89
2,Appropriations - S/C on Article II,https://capitol.texas.gov/Committees/MeetingsB...,H,89
3,Appropriations - S/C on Article III,https://capitol.texas.gov/Committees/MeetingsB...,H,89
4,"Appropriations - S/C on Articles I, IV, & V",https://capitol.texas.gov/Committees/MeetingsB...,H,89
5,"Appropriations - S/C on Articles VI, VII, & VIII",https://capitol.texas.gov/Committees/MeetingsB...,H,89
6,Calendars,https://capitol.texas.gov/Committees/MeetingsB...,H,89
7,Corrections,https://capitol.texas.gov/Committees/MeetingsB...,H,89
8,Criminal Jurisprudence,https://capitol.texas.gov/Committees/MeetingsB...,H,89
9,S/C on Juvenile Justice,https://capitol.texas.gov/Committees/MeetingsB...,H,89
